# Biblioteca Pandas

In [1]:
# importar biblioteca
import pandas as pd

In [2]:
local_arquivo = r'arquivos\petr4.csv'


Lendo arquivo csv local

In [3]:
df = pd.read_csv(local_arquivo, decimal=',')
df

,Data,Hora,Abertura,Maxima,Minima,Fechamento,ULT
0,21/05/2021,00:00:00,26.0800,26.3100,25.9500,26.0700,26.07
1,20/05/2021,00:00:00,26.1500,26.2800,25.7200,25.9300,26.07
2,19/05/2021,00:00:00,25.8500,26.3500,25.8000,26.1500,26.07
3,18/05/2021,00:00:00,26.7000,26.8000,26.2700,26.3500,26.07
4,17/05/2021,00:00:00,26.3000,26.6600,26.2000,26.6600,26.07
...,...,...,...,...,...,...,...
5157,18/07/2000,00:00:00,2.8343,2.8343,2.7603,2.7888,26.07
5158,17/07/2000,00:00:00,2.8457,2.9134,2.7546,2.8053,26.07
5159,14/07/2000,00:00:00,2.7888,2.8741,2.7774,2.8457,26.07
5160,13/07/2000,00:00:00,2.8741,2.8855,2.7575,2.7575,26.07


Tipo de objeto

In [4]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


Tipos de objetos dentro do dataframe

In [5]:
df.dtypes

Data           object
Hora           object
Abertura      float64
Maxima        float64
Minima        float64
Fechamento    float64
ULT           float64
dtype: object

Descrever o formato dos dados (linhas, colunas)

In [6]:
print(df.shape)

(5162, 7)


## Acesso à tabelas em html

datasets damodaran: http://pages.stern.nyu.edu/~adamodar/New_Home_Page/dataarchived.html

In [8]:
# TABELAS DA PÁGINA DO DAMODARAN
link = 'http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html'

df_html = pd.read_html(link)
df_html

[      0                                  1   \
 0    NaN  Annual Returns on  Investments in   
 1   Year       S&P 500 (includes dividends)   
 2   1928                             43.81%   
 3   1929                             -8.30%   
 4   1930                            -25.12%   
 ..   ...                                ...   
 90  2016                             11.77%   
 91  2017                             21.61%   
 92  2018                             -4.23%   
 93  2019                             31.21%   
 94  2020                             18.01%   
 
                                    2                                  3   \
 0   Annual Returns on  Investments in  Annual Returns on  Investments in   
 1                      3-month T.Bill                         US T. Bond   
 2                               3.08%                              0.84%   
 3                               3.16%                              4.20%   
 4                               4.55

Tipo do objeto

In [9]:
print(type(df_html))

<class 'list'>


## Tratando dados

Selecionando primeiro item da lista

In [10]:
df_html = df_html[0]
df_html

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NaN,Annual Returns on Investments in,Annual Returns on Investments in,Annual Returns on Investments in,Annual Returns on Investments in,Value of $100 invested at start of 1928 in,Value of $100 invested at start of 1928 in,Value of $100 invested at start of 1928 in,Value of $100 invested at start of 1928 in,Annual Risk Premium,Annual Risk Premium,Annual Risk Premium,Annual Risk Premium,NaN,Annual Real Returns on,Annual Real Returns on,Annual Real Returns on,Annual Real Returns on
1,Year,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,Baa Corporate Bond6,Stocks - Bills,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds
2,1928,43.81%,3.08%,0.84%,3.22%,$ 143.81,$ 103.08,$ 100.84,$ 103.22,40.73%,42.98%,40.59%,NaN,-1.15%,45.49%,4.28%,2.01%,4.42%
3,1929,-8.30%,3.16%,4.20%,3.02%,$ 131.88,$ 106.34,$ 105.07,$ 106.33,-11.46%,-12.50%,-11.32%,NaN,0.00%,-8.30%,3.16%,4.20%,3.02%
4,1930,-25.12%,4.55%,4.54%,0.54%,$ 98.75,$ 111.18,$ 109.85,$ 106.91,-29.67%,-29.66%,-25.66%,NaN,-2.67%,-23.07%,7.42%,7.41%,3.30%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2016,11.77%,0.32%,0.69%,10.37%,"$ 328,742.28","$ 1,990.70","$ 7,110.65","$ 39,552.74",11.46%,11.08%,1.41%,4.62%,1.26%,10.38%,-0.93%,-0.56%,8.99%
91,2017,21.61%,0.93%,2.80%,9.72%,"$ 399,768.64","$ 2,009.23","$ 7,309.87","$ 43,398.81",20.67%,18.80%,11.88%,4.77%,2.13%,19.07%,-1.17%,0.66%,7.44%
92,2018,-4.23%,1.94%,-0.02%,-2.76%,"$ 382,870.94","$ 2,048.20","$ 7,308.65","$ 42,199.86",-6.17%,-4.21%,-1.46%,4.66%,2.44%,-6.51%,-0.49%,-2.40%,-5.08%
93,2019,31.21%,1.55%,9.64%,15.33%,"$ 502,371.39","$ 2,079.94","$ 8,012.89","$ 48,668.87",29.66%,21.58%,15.88%,4.83%,2.29%,28.27%,-0.72%,7.18%,12.75%


Trocando o cabeçalho da coluna

In [11]:
df_html.columns = df_html.loc[1]
df_html

1,Year,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,Baa Corporate Bond6,Stocks - Bills,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds
0,NaN,Annual Returns on Investments in,Annual Returns on Investments in,Annual Returns on Investments in,Annual Returns on Investments in,Value of $100 invested at start of 1928 in,Value of $100 invested at start of 1928 in,Value of $100 invested at start of 1928 in,Value of $100 invested at start of 1928 in,Annual Risk Premium,Annual Risk Premium,Annual Risk Premium,Annual Risk Premium,NaN,Annual Real Returns on,Annual Real Returns on,Annual Real Returns on,Annual Real Returns on
1,Year,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,Baa Corporate Bond6,Stocks - Bills,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds
2,1928,43.81%,3.08%,0.84%,3.22%,$ 143.81,$ 103.08,$ 100.84,$ 103.22,40.73%,42.98%,40.59%,NaN,-1.15%,45.49%,4.28%,2.01%,4.42%
3,1929,-8.30%,3.16%,4.20%,3.02%,$ 131.88,$ 106.34,$ 105.07,$ 106.33,-11.46%,-12.50%,-11.32%,NaN,0.00%,-8.30%,3.16%,4.20%,3.02%
4,1930,-25.12%,4.55%,4.54%,0.54%,$ 98.75,$ 111.18,$ 109.85,$ 106.91,-29.67%,-29.66%,-25.66%,NaN,-2.67%,-23.07%,7.42%,7.41%,3.30%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2016,11.77%,0.32%,0.69%,10.37%,"$ 328,742.28","$ 1,990.70","$ 7,110.65","$ 39,552.74",11.46%,11.08%,1.41%,4.62%,1.26%,10.38%,-0.93%,-0.56%,8.99%
91,2017,21.61%,0.93%,2.80%,9.72%,"$ 399,768.64","$ 2,009.23","$ 7,309.87","$ 43,398.81",20.67%,18.80%,11.88%,4.77%,2.13%,19.07%,-1.17%,0.66%,7.44%
92,2018,-4.23%,1.94%,-0.02%,-2.76%,"$ 382,870.94","$ 2,048.20","$ 7,308.65","$ 42,199.86",-6.17%,-4.21%,-1.46%,4.66%,2.44%,-6.51%,-0.49%,-2.40%,-5.08%
93,2019,31.21%,1.55%,9.64%,15.33%,"$ 502,371.39","$ 2,079.94","$ 8,012.89","$ 48,668.87",29.66%,21.58%,15.88%,4.83%,2.29%,28.27%,-0.72%,7.18%,12.75%


Deletando colunas

In [13]:
df_html.drop([0, 1], inplace=True)
df_html

1,Year,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,Baa Corporate Bond6,Stocks - Bills,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds
2,1928,43.81%,3.08%,0.84%,3.22%,$ 143.81,$ 103.08,$ 100.84,$ 103.22,40.73%,42.98%,40.59%,NaN,-1.15%,45.49%,4.28%,2.01%,4.42%
3,1929,-8.30%,3.16%,4.20%,3.02%,$ 131.88,$ 106.34,$ 105.07,$ 106.33,-11.46%,-12.50%,-11.32%,NaN,0.00%,-8.30%,3.16%,4.20%,3.02%
4,1930,-25.12%,4.55%,4.54%,0.54%,$ 98.75,$ 111.18,$ 109.85,$ 106.91,-29.67%,-29.66%,-25.66%,NaN,-2.67%,-23.07%,7.42%,7.41%,3.30%
5,1931,-43.84%,2.31%,-2.56%,-15.68%,$ 55.46,$ 113.74,$ 107.03,$ 90.14,-46.15%,-41.28%,-28.16%,NaN,-8.93%,-38.33%,12.34%,7.00%,-7.41%
6,1932,-8.64%,1.07%,8.79%,23.59%,$ 50.66,$ 114.96,$ 116.44,$ 111.41,-9.71%,-17.43%,-32.23%,NaN,-10.30%,1.85%,12.68%,21.28%,37.78%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2016,11.77%,0.32%,0.69%,10.37%,"$ 328,742.28","$ 1,990.70","$ 7,110.65","$ 39,552.74",11.46%,11.08%,1.41%,4.62%,1.26%,10.38%,-0.93%,-0.56%,8.99%
91,2017,21.61%,0.93%,2.80%,9.72%,"$ 399,768.64","$ 2,009.23","$ 7,309.87","$ 43,398.81",20.67%,18.80%,11.88%,4.77%,2.13%,19.07%,-1.17%,0.66%,7.44%
92,2018,-4.23%,1.94%,-0.02%,-2.76%,"$ 382,870.94","$ 2,048.20","$ 7,308.65","$ 42,199.86",-6.17%,-4.21%,-1.46%,4.66%,2.44%,-6.51%,-0.49%,-2.40%,-5.08%
93,2019,31.21%,1.55%,9.64%,15.33%,"$ 502,371.39","$ 2,079.94","$ 8,012.89","$ 48,668.87",29.66%,21.58%,15.88%,4.83%,2.29%,28.27%,-0.72%,7.18%,12.75%


In [14]:
print(df_html.dtypes)

1
Year                              object
S&P 500 (includes dividends)      object
3-month T.Bill                    object
US T. Bond                        object
Baa  Corporate Bond               object
S&P 500 (includes dividends)3     object
3-month T.Bill4                   object
US T. Bond5                       object
Baa  Corporate Bond6              object
Stocks  - Bills                   object
Stocks  - Bonds                   object
Stocks  - Baa Corp Bond           object
Historical  risk premium          object
Inflation  Rate                   object
S&P  500 (includes dividends)2    object
3-month  T. Bill (Real)           object
!0-year  T.Bonds                  object
Baa  Corp Bonds                   object
dtype: object


Substituindo strings dentro do dataframe

In [15]:
df_html.replace('%', '', regex=True, inplace=True)
df_html.replace(',', '', regex=True, inplace=True)
df_html.replace('\$ ', '', regex=True, inplace=True)

Transformando objetos em dados numéricos

In [16]:
df_html = df_html.apply(pd.to_numeric)

In [17]:
df_html.dtypes

1
Year                                int64
S&P 500 (includes dividends)      float64
3-month T.Bill                    float64
US T. Bond                        float64
Baa  Corporate Bond               float64
S&P 500 (includes dividends)3     float64
3-month T.Bill4                   float64
US T. Bond5                       float64
Baa  Corporate Bond6              float64
Stocks  - Bills                   float64
Stocks  - Bonds                   float64
Stocks  - Baa Corp Bond           float64
Historical  risk premium          float64
Inflation  Rate                   float64
S&P  500 (includes dividends)2    float64
3-month  T. Bill (Real)           float64
!0-year  T.Bonds                  float64
Baa  Corp Bonds                   float64
dtype: object

Agora podemos fazer operações com o dataframe

In [18]:
df_html['S&P 500 (includes dividends)'] = df_html['S&P 500 (includes dividends)']/100

In [19]:
df_html

1,Year,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,Baa Corporate Bond6,Stocks - Bills,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds
2,1928,0.4381,3.08,0.84,3.22,143.81,103.08,100.84,103.22,40.73,42.98,40.59,NaN,-1.15,45.49,4.28,2.01,4.42
3,1929,-0.0830,3.16,4.20,3.02,131.88,106.34,105.07,106.33,-11.46,-12.50,-11.32,NaN,0.00,-8.30,3.16,4.20,3.02
4,1930,-0.2512,4.55,4.54,0.54,98.75,111.18,109.85,106.91,-29.67,-29.66,-25.66,NaN,-2.67,-23.07,7.42,7.41,3.30
5,1931,-0.4384,2.31,-2.56,-15.68,55.46,113.74,107.03,90.14,-46.15,-41.28,-28.16,NaN,-8.93,-38.33,12.34,7.00,-7.41
6,1932,-0.0864,1.07,8.79,23.59,50.66,114.96,116.44,111.41,-9.71,-17.43,-32.23,NaN,-10.30,1.85,12.68,21.28,37.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2016,0.1177,0.32,0.69,10.37,328742.28,1990.70,7110.65,39552.74,11.46,11.08,1.41,4.62,1.26,10.38,-0.93,-0.56,8.99
91,2017,0.2161,0.93,2.80,9.72,399768.64,2009.23,7309.87,43398.81,20.67,18.80,11.88,4.77,2.13,19.07,-1.17,0.66,7.44
92,2018,-0.0423,1.94,-0.02,-2.76,382870.94,2048.20,7308.65,42199.86,-6.17,-4.21,-1.46,4.66,2.44,-6.51,-0.49,-2.40,-5.08
93,2019,0.3121,1.55,9.64,15.33,502371.39,2079.94,8012.89,48668.87,29.66,21.58,15.88,4.83,2.29,28.27,-0.72,7.18,12.75


Transformar o "Year" em index

In [20]:
df_html.set_index('Year', inplace=True)
df_html

1,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,Baa Corporate Bond6,Stocks - Bills,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds
Year,,,,,,,,,,,,,,,,,
1928,0.4381,3.08,0.84,3.22,143.81,103.08,100.84,103.22,40.73,42.98,40.59,NaN,-1.15,45.49,4.28,2.01,4.42
1929,-0.0830,3.16,4.20,3.02,131.88,106.34,105.07,106.33,-11.46,-12.50,-11.32,NaN,0.00,-8.30,3.16,4.20,3.02
1930,-0.2512,4.55,4.54,0.54,98.75,111.18,109.85,106.91,-29.67,-29.66,-25.66,NaN,-2.67,-23.07,7.42,7.41,3.30
1931,-0.4384,2.31,-2.56,-15.68,55.46,113.74,107.03,90.14,-46.15,-41.28,-28.16,NaN,-8.93,-38.33,12.34,7.00,-7.41
1932,-0.0864,1.07,8.79,23.59,50.66,114.96,116.44,111.41,-9.71,-17.43,-32.23,NaN,-10.30,1.85,12.68,21.28,37.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,0.1177,0.32,0.69,10.37,328742.28,1990.70,7110.65,39552.74,11.46,11.08,1.41,4.62,1.26,10.38,-0.93,-0.56,8.99
2017,0.2161,0.93,2.80,9.72,399768.64,2009.23,7309.87,43398.81,20.67,18.80,11.88,4.77,2.13,19.07,-1.17,0.66,7.44
2018,-0.0423,1.94,-0.02,-2.76,382870.94,2048.20,7308.65,42199.86,-6.17,-4.21,-1.46,4.66,2.44,-6.51,-0.49,-2.40,-5.08


Ordenar pelo índice

In [21]:
df_html.sort_index(ascending=False, inplace=True)
df_html

1,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,Baa Corporate Bond6,Stocks - Bills,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds
Year,,,,,,,,,,,,,,,,,
2020,0.1801,0.09,11.33,10.41,592868.15,2081.82,8920.90,53736.05,17.92,6.68,7.60,4.84,1.20,16.61,-1.10,10.01,9.10
2019,0.3121,1.55,9.64,15.33,502371.39,2079.94,8012.89,48668.87,29.66,21.58,15.88,4.83,2.29,28.27,-0.72,7.18,12.75
2018,-0.0423,1.94,-0.02,-2.76,382870.94,2048.20,7308.65,42199.86,-6.17,-4.21,-1.46,4.66,2.44,-6.51,-0.49,-2.40,-5.08
2017,0.2161,0.93,2.80,9.72,399768.64,2009.23,7309.87,43398.81,20.67,18.80,11.88,4.77,2.13,19.07,-1.17,0.66,7.44
2016,0.1177,0.32,0.69,10.37,328742.28,1990.70,7110.65,39552.74,11.46,11.08,1.41,4.62,1.26,10.38,-0.93,-0.56,8.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1932,-0.0864,1.07,8.79,23.59,50.66,114.96,116.44,111.41,-9.71,-17.43,-32.23,NaN,-10.30,1.85,12.68,21.28,37.78
1931,-0.4384,2.31,-2.56,-15.68,55.46,113.74,107.03,90.14,-46.15,-41.28,-28.16,NaN,-8.93,-38.33,12.34,7.00,-7.41
1930,-0.2512,4.55,4.54,0.54,98.75,111.18,109.85,106.91,-29.67,-29.66,-25.66,NaN,-2.67,-23.07,7.42,7.41,3.30


Ordernar por outras colunas

In [22]:
df_html.sort_values(by='S&P 500 (includes dividends)', ascending=False, inplace=True)
df_html

1,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,Baa Corporate Bond6,Stocks - Bills,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds
Year,,,,,,,,,,,,,,,,,
1954,0.5256,0.94,3.29,6.16,783.18,132.17,218.57,412.31,51.62,49.27,46.41,NaN,0.31,52.09,0.63,2.97,5.83
1933,0.4998,0.96,1.86,12.97,75.99,116.06,118.60,125.86,49.02,48.13,37.02,NaN,-5.19,58.20,6.49,7.43,19.15
1935,0.4674,0.17,4.47,13.31,110.18,116.58,133.78,169.44,46.57,42.27,33.43,NaN,2.55,43.09,-2.33,1.87,10.49
1928,0.4381,3.08,0.84,3.22,143.81,103.08,100.84,103.22,40.73,42.98,40.59,NaN,-1.15,45.49,4.28,2.01,4.42
1958,0.4372,1.77,-2.10,6.43,1435.84,144.95,220.39,434.11,41.95,45.82,37.29,NaN,2.73,39.90,-0.93,-4.70,3.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,-0.2512,4.55,4.54,0.54,98.75,111.18,109.85,106.91,-29.67,-29.66,-25.66,NaN,-2.67,-23.07,7.42,7.41,3.30
1974,-0.2590,7.83,1.99,-4.38,3023.54,295.08,379.98,806.95,-33.73,-27.89,-21.52,4.64,11.05,-33.28,-2.90,-8.16,-13.90
1937,-0.3534,0.28,1.38,-4.42,94.00,117.11,142.43,180.39,-35.61,-36.72,-30.92,NaN,3.73,-37.66,-3.33,-2.26,-7.85


## Código completo read_html

In [23]:
# TABELAS DA PÁGINA DO DAMODARAN
link = 'http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html'

# Lê a tabela e já seleciona o primeiro item da lista de dataframes
df_html = pd.read_html(link)[0]

# Troca os títulos das colunas para a segunda linha do dataframe
df_html.columns = df_html.loc[1]

# Apaga a linha 0 e 1 do dataframe pois são iguais aos títulos
df_html.drop([0, 1], inplace=True)

# Substituindo strings que atrapalham a identificação correta dos tipos de objeto
df_html.replace('%', '', regex=True, inplace=True)
df_html.replace(',', '', regex=True, inplace=True)
df_html.replace('\$ ', '', regex=True, inplace=True)

# Transformando em objetos do tipo numérico
df_html = df_html.apply(pd.to_numeric)

# Dividindo a coluna da rentabilidade por 100 para ficar em porcentagem
df_html['S&P 500 (includes dividends)'] = df_html['S&P 500 (includes dividends)']/100

# Usando a data como índice
df_html['Year'] = pd.to_datetime(df_html['Year'], format='%Y')
df_html.set_index('Year', inplace=True)

df_html

URLError: <urlopen error [WinError 10060] Uma tentativa de conexão falhou porque o componente conectado não respondeu
corretamente após um período de tempo ou a conexão estabelecida falhou
porque o host conectado não respondeu>

## Trabalhando com datetime

In [24]:
arquivo = r'arquivos\ibov.csv'
nome_papel = arquivo.split('\\')[-1][:-4]
nome_papel = nome_papel.upper()
print(nome_papel)
df = pd.read_csv(arquivo, decimal=',', thousands='.')
df

IBOV


,Data,Hora,Abertura,Maxima,Minima,Fechamento,ULT
0,21/05/2021,00:00:00,122701.4400,122799.3600,121760.3700,122618.1900,122618.19
1,20/05/2021,00:00:00,122635.9375,122733.9531,122136.1562,122700.7891,122618.19
2,19/05/2021,00:00:00,122975.9297,123013.3984,121595.2734,122636.2969,122618.19
3,18/05/2021,00:00:00,122937.4219,123543.6797,122549.7891,122979.9609,122618.19
4,17/05/2021,00:00:00,121880.2812,123074.2109,121680.4688,122937.8672,122618.19
...,...,...,...,...,...,...,...
5279,06/01/2000,00:00:00,16237.0000,16498.0000,15977.0000,16106.0000,122618.19
5280,05/01/2000,00:00:00,15871.0000,16302.0000,15349.0000,16245.0000,122618.19
5281,04/01/2000,00:00:00,16907.0000,16907.0000,15851.0000,15851.0000,122618.19
5282,03/01/2000,00:00:00,17098.0000,17408.0000,16719.0000,16930.0000,122618.19


Apagando as colunas que não serão usadas

In [25]:
df.drop(columns=['Hora', 'ULT'], inplace=True)

Transformar a data em datetime

In [26]:
df.dtypes

Data           object
Abertura      float64
Maxima        float64
Minima        float64
Fechamento    float64
dtype: object

In [27]:
df['Data_teste'] = pd.to_datetime(df['Data'])
df.dtypes

Data                  object
Abertura             float64
Maxima               float64
Minima               float64
Fechamento           float64
Data_teste    datetime64[ns]
dtype: object

Conferindo formato da data

aaaa-mm-dd

In [28]:
df.sort_values(by='Data_teste', ascending=False)

,Data,Abertura,Maxima,Minima,Fechamento,Data_teste
7,12/05/2021,122963.9609,122963.9609,119458.4297,119710.0312,2021-12-05
28,12/04/2021,117660.7734,118811.7422,117660.7734,118811.7422,2021-12-04
48,12/03/2021,114983.7578,114983.7578,113253.1016,114160.3984,2021-12-03
66,12/02/2021,119299.8281,119763.3594,118162.6797,119428.7188,2021-12-02
88,12/01/2021,123255.1328,124584.3281,123227.4688,123998.0000,2021-12-01
...,...,...,...,...,...,...
5137,01/08/2000,16474.0000,16591.0000,16124.0000,16289.0000,2000-01-08
5179,01/06/2000,14967.0000,15472.0000,14967.0000,15450.0000,2000-01-06
5241,01/03/2000,17668.0000,17953.0000,17654.0000,17953.0000,2000-01-03
5262,01/02/2000,16389.0000,16675.0000,16389.0000,16522.0000,2000-01-02


In [29]:
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')
df.sort_values(by='Data', ascending=False, inplace=True)

In [30]:
df.drop(columns='Data_teste', inplace=True)
df

,Data,Abertura,Maxima,Minima,Fechamento
0,2021-05-21,122701.4400,122799.3600,121760.3700,122618.1900
1,2021-05-20,122635.9375,122733.9531,122136.1562,122700.7891
2,2021-05-19,122975.9297,123013.3984,121595.2734,122636.2969
3,2021-05-18,122937.4219,123543.6797,122549.7891,122979.9609
4,2021-05-17,121880.2812,123074.2109,121680.4688,122937.8672
...,...,...,...,...,...
5279,2000-01-06,16237.0000,16498.0000,15977.0000,16106.0000
5280,2000-01-05,15871.0000,16302.0000,15349.0000,16245.0000
5281,2000-01-04,16907.0000,16907.0000,15851.0000,15851.0000
5282,2000-01-03,17098.0000,17408.0000,16719.0000,16930.0000


Usando a data como índice

In [31]:
df.set_index('Data', inplace=True)

Transformando em dados mensais

In [32]:
df_mensal = df.resample('M').last()
df_mensal

,Abertura,Maxima,Minima,Fechamento
Data,,,,
1999-12-31,16777.0000,17104.0000,16777.0000,17091.0000
2000-01-31,16743.0000,16748.0000,16167.0000,16388.0000
2000-02-29,17544.0000,17822.0000,17544.0000,17660.0000
2000-03-31,17659.0000,17949.0000,17559.0000,17820.0000
2000-04-30,15450.0000,15906.0000,15450.0000,15537.0000
...,...,...,...,...
2021-01-31,118879.8984,118879.8984,114973.0000,115067.5469
2021-02-28,112260.2109,113466.2109,109827.0938,110035.1719
2021-03-31,116849.6719,117248.5078,115932.4219,116633.7188


In [33]:
df.rename(columns={'Fechamento': nome_papel}, inplace=True)
df

,Abertura,Maxima,Minima,IBOV
Data,,,,
2021-05-21,122701.4400,122799.3600,121760.3700,122618.1900
2021-05-20,122635.9375,122733.9531,122136.1562,122700.7891
2021-05-19,122975.9297,123013.3984,121595.2734,122636.2969
2021-05-18,122937.4219,123543.6797,122549.7891,122979.9609
2021-05-17,121880.2812,123074.2109,121680.4688,122937.8672
...,...,...,...,...
2000-01-06,16237.0000,16498.0000,15977.0000,16106.0000
2000-01-05,15871.0000,16302.0000,15349.0000,16245.0000
2000-01-04,16907.0000,16907.0000,15851.0000,15851.0000


In [34]:
df_anual = df.resample('Y').last()
df_anual

,Abertura,Maxima,Minima,IBOV
Data,,,,
1999-12-31,16777.0000,17104.0000,16777.0000,17091.0000
2000-12-31,15188.0000,15269.0000,15132.0000,15259.0000
2001-12-31,13741.0000,13960.0000,13509.0000,13577.0000
2002-12-31,11235.0000,11324.0000,11211.0000,11268.0000
2003-12-31,22050.0000,22310.0000,22050.0000,22236.0000
2004-12-31,26170.0000,26269.0000,26108.0000,26196.0000
2005-12-31,33137.0000,33455.0000,33137.0000,33455.0000
2006-12-31,44526.0000,44674.0000,44372.0000,44473.0000
2007-12-31,63776.0000,64123.0000,63598.0000,63886.0000


Calcular o retorno anual

In [35]:
df_anual[f'retorno_{nome_papel}'] = df_anual[nome_papel].pct_change()
df_anual

,Abertura,Maxima,Minima,IBOV,retorno_IBOV
Data,,,,,
1999-12-31,16777.0000,17104.0000,16777.0000,17091.0000,NaN
2000-12-31,15188.0000,15269.0000,15132.0000,15259.0000,-0.107191
2001-12-31,13741.0000,13960.0000,13509.0000,13577.0000,-0.110230
2002-12-31,11235.0000,11324.0000,11211.0000,11268.0000,-0.170067
2003-12-31,22050.0000,22310.0000,22050.0000,22236.0000,0.973376
2004-12-31,26170.0000,26269.0000,26108.0000,26196.0000,0.178090
2005-12-31,33137.0000,33455.0000,33137.0000,33455.0000,0.277103
2006-12-31,44526.0000,44674.0000,44372.0000,44473.0000,0.329338
2007-12-31,63776.0000,64123.0000,63598.0000,63886.0000,0.436512


## Juntando dois dataframes

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [36]:
df_html

1,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,Baa Corporate Bond6,Stocks - Bills,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds
Year,,,,,,,,,,,,,,,,,
1954,0.5256,0.94,3.29,6.16,783.18,132.17,218.57,412.31,51.62,49.27,46.41,NaN,0.31,52.09,0.63,2.97,5.83
1933,0.4998,0.96,1.86,12.97,75.99,116.06,118.60,125.86,49.02,48.13,37.02,NaN,-5.19,58.20,6.49,7.43,19.15
1935,0.4674,0.17,4.47,13.31,110.18,116.58,133.78,169.44,46.57,42.27,33.43,NaN,2.55,43.09,-2.33,1.87,10.49
1928,0.4381,3.08,0.84,3.22,143.81,103.08,100.84,103.22,40.73,42.98,40.59,NaN,-1.15,45.49,4.28,2.01,4.42
1958,0.4372,1.77,-2.10,6.43,1435.84,144.95,220.39,434.11,41.95,45.82,37.29,NaN,2.73,39.90,-0.93,-4.70,3.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,-0.2512,4.55,4.54,0.54,98.75,111.18,109.85,106.91,-29.67,-29.66,-25.66,NaN,-2.67,-23.07,7.42,7.41,3.30
1974,-0.2590,7.83,1.99,-4.38,3023.54,295.08,379.98,806.95,-33.73,-27.89,-21.52,4.64,11.05,-33.28,-2.90,-8.16,-13.90
1937,-0.3534,0.28,1.38,-4.42,94.00,117.11,142.43,180.39,-35.61,-36.72,-30.92,NaN,3.73,-37.66,-3.33,-2.26,-7.85


In [37]:
df_sp = df_html.copy()
df_sp

1,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,Baa Corporate Bond6,Stocks - Bills,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds
Year,,,,,,,,,,,,,,,,,
1954,0.5256,0.94,3.29,6.16,783.18,132.17,218.57,412.31,51.62,49.27,46.41,NaN,0.31,52.09,0.63,2.97,5.83
1933,0.4998,0.96,1.86,12.97,75.99,116.06,118.60,125.86,49.02,48.13,37.02,NaN,-5.19,58.20,6.49,7.43,19.15
1935,0.4674,0.17,4.47,13.31,110.18,116.58,133.78,169.44,46.57,42.27,33.43,NaN,2.55,43.09,-2.33,1.87,10.49
1928,0.4381,3.08,0.84,3.22,143.81,103.08,100.84,103.22,40.73,42.98,40.59,NaN,-1.15,45.49,4.28,2.01,4.42
1958,0.4372,1.77,-2.10,6.43,1435.84,144.95,220.39,434.11,41.95,45.82,37.29,NaN,2.73,39.90,-0.93,-4.70,3.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,-0.2512,4.55,4.54,0.54,98.75,111.18,109.85,106.91,-29.67,-29.66,-25.66,NaN,-2.67,-23.07,7.42,7.41,3.30
1974,-0.2590,7.83,1.99,-4.38,3023.54,295.08,379.98,806.95,-33.73,-27.89,-21.52,4.64,11.05,-33.28,-2.90,-8.16,-13.90
1937,-0.3534,0.28,1.38,-4.42,94.00,117.11,142.43,180.39,-35.61,-36.72,-30.92,NaN,3.73,-37.66,-3.33,-2.26,-7.85


Selecionando as primeiras colunas

In [38]:
df_sp = df_sp[df_sp.columns[0:3]]
df_sp

1,S&P 500 (includes dividends),3-month T.Bill,US T. Bond
Year,,,
1954,0.5256,0.94,3.29
1933,0.4998,0.96,1.86
1935,0.4674,0.17,4.47
1928,0.4381,3.08,0.84
1958,0.4372,1.77,-2.10
...,...,...,...
1930,-0.2512,4.55,4.54
1974,-0.2590,7.83,1.99
1937,-0.3534,0.28,1.38


Transformar a data e ano em apenas ano

In [39]:
df_sp.index = df_sp.index.year
df_sp

AttributeError: 'Int64Index' object has no attribute 'year'

In [ ]:
df_anual.index = df_anual.index.year
df_anual

In [ ]:
df_junto = df_anual.merge(df_sp, right_index=True, left_index=True)

df_junto.dropna(inplace=True)

df_junto = df_junto[['retorno_IBOV', 'S&P 500 (includes dividends)']]
df_junto

In [ ]:
df_junto.plot()

Baixando dados de um link da b3

In [ ]:
# Retornos do IBOV
link_ibov = 'http://bvmf.bmfbovespa.com.br/indices/ResumoVariacaoAnual.aspx?Indice=IBOV&idioma=pt-br'
df_online = pd.read_html(link_ibov, decimal=',', thousands='.')[0].dropna()

# Limpando o dataframe
df_online.replace('\*', '', regex=True, inplace=True)
df_online.replace(' \(\)', '', regex=True, inplace=True)

# Transformando o ano em índice
df_online['Ano'] = pd.to_datetime(df_online['Ano'], format='%Y')
df_online.set_index('Ano', inplace=True)
df_online.index = df_online.index.year

# Transformando em %
df_online[['Variação anual em US$', 'Variação Anual Nominal']] = df_online[['Variação anual em US$', 'Variação Anual Nominal']]/100
df_online['conta'] = df_online['Variação anual em US$'] / df_online['Variação Anual Nominal']
df_online['Variação Anual Nominal'].mean()

Juntando o novo dataframe

In [ ]:
df_junto = df_junto.merge(df_online[['Variação anual em US$']], right_index=True, left_index=True)
df_junto

Plotando gráfico

In [ ]:
df_junto[['S&P 500 (includes dividends)', 'Variação anual em US$']].plot()

Descrevendo os dados

In [ ]:
df_junto.describe()

## Criando a função + Código completo retorno

In [40]:
def dados_retornos(arquivo, freq='Y'):
    
    # Nome do ativo do arquivo
    nome_papel = arquivo.split('\\')[-1][:-4]
    nome_papel = nome_papel.upper()
    
    # Ler o arquivo em csv
    df = pd.read_csv(arquivo, decimal=',', thousands='.')
    
    # Dropando colunas não usadas
    df.drop(columns=['Hora', 'ULT', 'Abertura', 'Maxima', 'Minima'], inplace=True)
    
    # Trocando o nome 'Fechamento' para o nome do ativo
    df.rename(columns={'Fechamento': nome_papel}, inplace=True)

    # Transformando a data no formato correto de datetime
    df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')
    
    # Usando a coluna Data como índice
    df.set_index('Data', inplace=True)
    
    # Frequência dos dados 'M', 'Q', 'Y'
    df = df.resample(freq).last()
    
    # Ordenando por data menor para maior
    df.sort_index(ascending=True, inplace=True)
    
    # Calculando o retorno no período
    df[f'retorno_{nome_papel}'] = df[nome_papel].pct_change()
    
    # Apagando linhas onde existem dados em branco
    df.dropna(inplace=True)
    
    return df

In [41]:
arquivo = r'arquivos\ibov.csv'
retorno_papel = dados_retornos(arquivo, 'Y')
retorno_papel

,IBOV,retorno_IBOV
Data,,
2000-12-31,15259.0000,-0.107191
2001-12-31,13577.0000,-0.110230
2002-12-31,11268.0000,-0.170067
2003-12-31,22236.0000,0.973376
2004-12-31,26196.0000,0.178090
2005-12-31,33455.0000,0.277103
2006-12-31,44473.0000,0.329338
2007-12-31,63886.0000,0.436512
2008-12-31,37550.0000,-0.412234


Juntando todos arquivos baixados

In [43]:
# Pacote que faz iteração com arquivos e pastas
import glob

# Local onde se encontram os arquivos \ tipo do arquivo
path = 'arquivos'
todos_arquivos = glob.glob(path+'\*.csv')

# Criar um dataframe em branco
geral = pd.DataFrame()

# Loop executando nossa função e adicionando ao dataframe em branco 
for local in todos_arquivos:
    
    print(local)
    
    df = dados_retornos(local, 'M')
    geral = pd.concat([geral, df], axis=1)

geral

arquivos\abev3.csv
arquivos\ibov.csv
arquivos\petr4.csv
arquivos\vale3.csv


,ABEV3,retorno_ABEV3,IBOV,retorno_IBOV,PETR4,retorno_PETR4,VALE3,retorno_VALE3
Data,,,,,,,,
2000-01-31,NaN,NaN,16388.0000,-0.041133,NaN,NaN,NaN,NaN
2000-02-29,NaN,NaN,17660.0000,0.077618,NaN,NaN,NaN,NaN
2000-03-31,NaN,NaN,17820.0000,0.009060,NaN,NaN,NaN,NaN
2000-04-30,NaN,NaN,15537.0000,-0.128114,NaN,NaN,NaN,NaN
2000-05-31,NaN,NaN,14956.0000,-0.037395,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2021-01-31,15.11,-0.029905,115067.5469,-0.033186,25.8231,-0.058221,84.2921,0.005717
2021-02-28,14.02,-0.072138,110035.1719,-0.043734,21.5177,-0.166727,90.5888,0.074701
2021-03-31,15.29,0.090585,116633.7188,0.059968,23.3172,0.083629,97.9700,0.081480


Apagando as linhas onde há dados em branco

In [ ]:
geral.dropna(inplace=True)
geral

Separando listas de colunas do tipo retorno e fechamento

In [ ]:
colunas_retorno = [coluna for coluna in geral.columns if 'retorno' in coluna]
colunas_fechamento = [coluna for coluna in geral.columns if 'retorno' not in coluna]
colunas_retorno

In [ ]:
geral[colunas_fechamento].drop(columns=['IBOV']).plot()

In [ ]:
geral[colunas_retorno].describe()